In [1]:
# 案
# misconceptionを1件見せ、それに合うようなhoge, fuga, piyoを生成してください
# 例を5つ(ランダムに選択)与えるため、参考にしてください。

In [2]:
# まずは、どのようなpromptを投げたら、良い感じの誤解に基づいた問題を生成してくれるかを調べる。
import polars as pl

In [3]:
EXP_NAME = "e001-generate"
DATA_PATH = "data"
ENV_PATH = "env_file"
DATASET_NAME = EXP_NAME
COMPETITION_NAME = "eedi-mining-misconceptions-in-mathematics"
OUTPUT_PATH = f"output/generate_data/{EXP_NAME}"

DEBUG = False
UPLOAD_DATA_TO_S3 = True
UPLOAD_DATA_TO_KAGGLE = True

In [4]:
def resolve_path(base_path: str) -> str:
    import os

    cwd = os.getcwd()
    print(cwd)
    if cwd == f"/notebooks":
        print("Jupyter Kernel By VSCode!")
        return f"/notebooks/{COMPETITION_NAME}/{base_path}"
    elif cwd == f"/notebooks/{COMPETITION_NAME}":
        print("nohup!")
        return base_path
    elif cwd == f"/notebooks/{COMPETITION_NAME}/{COMPETITION_NAME}/exp":
        print("Jupyter Lab!")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/reranker":
        print("VastAi! Reranker")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/retriever":
        print("VastAi! Retriever")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/causal":
        print("VastAi! Causal")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/generate_data":
        print("VastAi! Generate Data")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}":
        print("VastAi!")
        return base_path
    else:
        raise Exception("Unknown environment")


DATA_PATH = resolve_path(DATA_PATH)
print(DATA_PATH)
ENV_PATH = resolve_path(ENV_PATH)
print(ENV_PATH)
OUTPUT_PATH = resolve_path(OUTPUT_PATH)
print(OUTPUT_PATH)

In [5]:
def validate_dataset_name(dataset_name: str) -> None:
    if len(dataset_name) < 6 or len(dataset_name) > 50:
        raise Exception(
            f"データセットの文字列は6~50文字にしてください。現在{len(DATASET_NAME)}文字"
        )
    if "_" in dataset_name:
        raise Exception("datasetの名称に_の使用は禁止です")


validate_dataset_name(DATASET_NAME)

In [6]:
train = pl.read_csv(f"{DATA_PATH}/train_long.csv")
misconception_mapping = pl.read_csv(f"{DATA_PATH}/misconception_mapping.csv")

In [7]:
# MisconceptionNameがnullではない部分だけを残す
train = train.filter(pl.col("MisconceptionName").is_not_null())

In [8]:
PROMPT_TEMPLATE = """I will provide Misconception. 
Please create a misleading question from misconception.
Include in the problem: Question, Construct Name, Subject Name, Correct Answer, Incorrect Answer, Misconception Name.
Please make sure that Misconception Name is always exactly the same as the Misconception that was initially provided.
I will provide 3 examples, so please use them as a reference.

### Misconception
{make_misconception_name}

### Example1
Question: {question_text_1}
Construct Name: {construct_name_1}
Subject Name: {subject_name_1}

Correct Answer: {correct_answer_1}
Incorrect Answer: {incorrect_answer_1}

Misconception Name: {misconception_name_1}

### Example2
Question: {question_text_2}
Construct Name: {construct_name_2}
Subject Name: {subject_name_2}

Correct Answer: {correct_answer_2}
Incorrect Answer: {incorrect_answer_2}

Misconception Name: {misconception_name_2}

### Example3
Question: {question_text_3}
Construct Name: {construct_name_3}
Subject Name: {subject_name_3}

Correct Answer: {correct_answer_3}
Incorrect Answer: {incorrect_answer_3}

Misconception Name: {misconception_name_3}
"""


def make_prompt(df: pl.DataFrame, make_misconception_name: str) -> str:
    row = train.sample(3)

    question_text_1 = row["QuestionText"][0]
    construct_name_1 = row["ConstructName"][0]
    subject_name_1 = row["SubjectName"][0]
    correct_answer_1 = row["CorrectAnswerText"][0]
    incorrect_answer_1 = row["AnswerText"][0]
    misconception_name_1 = row["MisconceptionName"][0]

    question_text_2 = row["QuestionText"][1]
    construct_name_2 = row["ConstructName"][1]
    subject_name_2 = row["SubjectName"][1]
    correct_answer_2 = row["CorrectAnswerText"][1]
    incorrect_answer_2 = row["AnswerText"][1]
    misconception_name_2 = row["MisconceptionName"][1]

    question_text_3 = row["QuestionText"][2]
    construct_name_3 = row["ConstructName"][2]
    subject_name_3 = row["SubjectName"][2]
    correct_answer_3 = row["CorrectAnswerText"][2]
    incorrect_answer_3 = row["AnswerText"][2]
    misconception_name_3 = row["MisconceptionName"][2]

    prompt = PROMPT_TEMPLATE.format(
        make_misconception_name=make_misconception_name,
        question_text_1=question_text_1,
        construct_name_1=construct_name_1,
        subject_name_1=subject_name_1,
        correct_answer_1=correct_answer_1,
        incorrect_answer_1=incorrect_answer_1,
        misconception_name_1=misconception_name_1,
        question_text_2=question_text_2,
        construct_name_2=construct_name_2,
        subject_name_2=subject_name_2,
        correct_answer_2=correct_answer_2,
        incorrect_answer_2=incorrect_answer_2,
        misconception_name_2=misconception_name_2,
        question_text_3=question_text_3,
        construct_name_3=construct_name_3,
        subject_name_3=subject_name_3,
        correct_answer_3=correct_answer_3,
        incorrect_answer_3=incorrect_answer_3,
        misconception_name_3=misconception_name_3,
    )

    print(prompt)
    return prompt


make_prompt(train, misconception_mapping["MisconceptionName"][0])

'I will provide Misconception. \nPlease create a misleading question from misconception.\nInclude in the problem: Question, Construct Name, Subject Name, Correct Answer, Incorrect Answer, Misconception Name.\nPlease make sure that Misconception Name is always exactly the same as the Misconception that was initially provided.\nI will provide 3 examples, so please use them as a reference.\n\n### Misconception\nDoes not know that angles in a triangle sum to 180 degrees\n\n### Example1\nQuestion: How would you describe the movement to get from the star to the triangle? ![A set of axes: x-axis from 0 to 5, y-axis from 0 to 5. A purple triangle symbol is at point (2,1) and a yellow star symbol is at point (1, 4).]()\nConstruct Name: Describe simple directions to translate a shape or object using words not vectors\nSubject Name: Translation and Vectors\n\nCorrect Answer: \\( 1 \\) squares right, \\( 3 \\) squares down\nIncorrect Answer: \\( 1 \\) squares right,\\( 4 \\) squares down\n\nMiscon

# Make Data By API

In [9]:
from dotenv import load_dotenv

load_dotenv(f"{ENV_PATH}/.env")

True

In [10]:
from openai import OpenAI

client = OpenAI()


def get_openai_result(text: str) -> str:
    return (
        client.chat.completions.create(
            model="gpt-4o",
            # model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": "You are a mathematics expert. You are given Misconception. Please create a misleading question from Misconception. Include in the problem: ConstructName, SubjectName, QuestionText, CorrectAnswer, AnswerText. I will provide 3 examples, so please use them as a reference.",
                },
                {"role": "user", "content": text},
            ],
        )
        .choices[0]
        .message.content
    )

In [11]:
import re

key_list = [
    "Question",
    "Construct Name",
    "Subject Name",
    "Correct Answer",
    "Incorrect Answer",
    "Misconception Name",
]


def get_list_from_res(res_text: str) -> list[str]:
    return [
        re.search(rf"{key}: (.*?)(\n\n|\n|$)", res_text).group(1) for key in key_list
    ]

In [12]:
from tqdm.auto import tqdm

gen_data_result = []

gen_misconception_names = (
    misconception_mapping["MisconceptionName"]
    if not DEBUG
    else misconception_mapping["MisconceptionName"][:10]
)

for misconception_name in tqdm(
    gen_misconception_names,
    total=misconception_mapping.shape[0],
):
    try:
        res_text = get_openai_result(make_prompt(train, misconception_name))
        res_list = get_list_from_res(res_text)
        gen_data_result.append(res_list)
    except:
        print("Error")
        pass

  0%|          | 0/2587 [00:00<?, ?it/s]

In [13]:
import os

os.makedirs(OUTPUT_PATH, exist_ok=True)

pl.DataFrame(
    gen_data_result,
    schema=[
        "QuestionText",
        "ConstructName",
        "SubjectName",
        "CorrectAnswerText",
        "IncorrectAnswerText",
        "MisconceptionName",
    ],
).write_csv(f"{OUTPUT_PATH}/{EXP_NAME}.csv")

In [14]:
gen_data_result

[['If you have a triangle with two angles measuring \\(60^\\circ\\) and \\(80^\\circ\\), what is the measure of the third angle in the triangle?',
  'Use the angle sum property of triangles to find a missing angle',
  'Geometry',
  '\\(40^\\circ\\)',
  '\\(50^\\circ\\)',
  'Does not know that angles in a triangle sum to 180 degrees'],
 ['What is the product of \\( \\frac{1}{3} \\) and \\( \\frac{4}{5} \\)?',
  'Multiply fractions by applying the correct multiplication rule',
  'Fractions',
  '\\( \\frac{4}{15} \\)',
  '\\( \\frac{5}{12} \\)',
  'Uses dividing fractions method for multiplying fractions'],
 ["Sarah is designing a pinwheel and wants one of the segments to span one-fifth of a full turn. She calculates this angle, but isn't confident in her result. What angle should she use?",
  'Understand and use angle measurement in degrees',
  'Geometry',
  '72 degrees',
  '20 degrees',
  'Believes there are 100 degrees in a full turn'],
 ['Consider the quadratic equation \\( x^2 - 4x \

In [15]:
print(pl.DataFrame(gen_data_result, schema=key_list).shape)

# AWSへのアップロード

In [16]:
# S3へのアップロード
if not DEBUG and UPLOAD_DATA_TO_S3:
    # uninstall
    !sudo rm /usr/bin/aws
    !sudo rm /usr/bin/aws_completer
    !sudo rm -rf /usr/local/aws-cli

    # install
    !curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
    !unzip -o -qq awscliv2.zip
    !sudo ./aws/install --update

    # upload
    output_name = OUTPUT_PATH.split("/")[-1]
    os.system(
        f"aws s3 cp --recursive {OUTPUT_PATH} s3://{COMPETITION_NAME}/trained_model/{output_name}"
    )

# Kaggle Upload

In [17]:
import os
import json

from kaggle.api.kaggle_api_extended import KaggleApi


def dataset_create_new(dataset_name: str, upload_dir: str):
    # if "_" in dataset_name:
    #     raise ValueError("datasetの名称に_の使用は禁止です")
    dataset_metadata = {}
    dataset_metadata["id"] = f"sinchir0/{dataset_name}"
    dataset_metadata["licenses"] = [{"name": "CC0-1.0"}]
    dataset_metadata["title"] = dataset_name
    with open(os.path.join(upload_dir, "dataset-metadata.json"), "w") as f:
        json.dump(dataset_metadata, f, indent=4)
    api = KaggleApi()
    api.authenticate()
    api.dataset_create_new(folder=upload_dir, convert_to_csv=False, dir_mode="tar")


print(f"Create Dataset name:{DATASET_NAME}, output_dir:{OUTPUT_PATH}")
dataset_create_new(dataset_name=DATASET_NAME, upload_dir=OUTPUT_PATH)